In [48]:
from pprint import pprint
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
db = client.db_agg
restaurants = db.restaurants
transactions = db.transactions
print('connexion success')

connexion success


#  Exercice  Aggréger des restaurants 

In [19]:
# Q : Créer 1DB restaurant, ajouter 1 collection "restaurants", la remplir avec le fichier restaurants.json.


#commande mongo:
# mongoimport -d restaurant -c restaurants --drop --file "C:\Users\vince\Downloads\NoSQL\fichierJson\restaurants.json"


In [20]:
# Q : La répartition des restaurants par quartier ?

repart_resto_borough = { "$group":{"_id": "$borough","nb_restaurant":{"$sum" : 1}}}
tri = {"$sort" : {"_id":1}}
for resto in restaurants.aggregate([repart_resto_borough, tri]):
    print(resto)


{'_id': 'Bronx', 'nb_restaurant': 2338}
{'_id': 'Brooklyn', 'nb_restaurant': 6085}
{'_id': 'Manhattan', 'nb_restaurant': 10258}
{'_id': 'Missing', 'nb_restaurant': 51}
{'_id': 'Queens', 'nb_restaurant': 5656}
{'_id': 'Staten Island', 'nb_restaurant': 969}


In [21]:
# Q : La répartition des restaurants dont la dernière inspection a donné un grade "C" par quartier ?

match_grade_C = { "$match" : {"grades.0.grade":"C"}}
proj = {"$project" : {"name":1, "borough":1,"_id":0}}
tri = {"$sort" : {"_id":1}}
for resto in restaurants.aggregate([match_grade_C,proj,repart_resto_borough, tri]):
    print(resto)

{'_id': 'Bronx', 'nb_restaurant': 27}
{'_id': 'Brooklyn', 'nb_restaurant': 56}
{'_id': 'Manhattan', 'nb_restaurant': 83}
{'_id': 'Queens', 'nb_restaurant': 47}
{'_id': 'Staten Island', 'nb_restaurant': 7}


In [22]:
# Q : Calculer le score moyen des resto par quartier et trier par score décroissant ?

eclater_liste = {"$unwind" : "$grades"}
mean_quartier = {"$group" : {"_id":"$borough","moyenne":{"$avg":"$grades.score"}}}
tri = {"$sort": {"moyenne":-1}}
for restaurant in restaurants.aggregate([eclater_liste, mean_quartier, tri]):
    print(restaurant)

{'_id': 'Queens', 'moyenne': 11.634865110930088}
{'_id': 'Brooklyn', 'moyenne': 11.447723132969035}
{'_id': 'Manhattan', 'moyenne': 11.41823125728344}
{'_id': 'Staten Island', 'moyenne': 11.370957711442786}
{'_id': 'Bronx', 'moyenne': 11.036186099942562}
{'_id': 'Missing', 'moyenne': 9.632911392405063}


# Exercice  Aggréger des transactions  


In [29]:
# Q : Créer une collection "transactions" à partir du fichier transactions.json et répondre aux questions suivantes :

#commande mongo :
# mongoimport -d db_agg -c transactions --drop --file "C:\Users\vince\Downloads\NoSQL\fichierJson\transactions.json" --jsonArray


In [79]:
# Q-1. Calculer le montant total des paiements ?

match = {"$match" : {"Payment.Success":True}}
group = {"$group" : {"_id":"Total","Total":{"$sum":"$Payment.Total"}}}
for transaction in transactions.aggregate([match, group]):
    print(transaction)

{'_id': 'Total', 'Total': 1230}


In [76]:
# Q-2. Calculer le montant total par paiment ?

eclater_liste = {"$unwind" : "$Panier"}
group = {"$group" : {"_id":"$TransactionId","Total":{"$sum":"$Panier.price"}}}
for transaction in transactions.aggregate([eclater_liste,group]):
    pprint(transaction)


{'Total': 410, '_id': 'tran3'}
{'Total': 410, '_id': 'tran4'}
{'Total': 410, '_id': 'tran1'}
{'Total': 410, '_id': 'tran2'}


In [63]:
# Q-3. Calculate total payments (Payment.Total) for each payment type (Payment.Type) ?

match = {"$match" : {"Payment.Success":True}}
group = {"$group" : {"_id":"$Payment.Type","Total":{"$sum":"$Payment.Total"}}}
for transaction in transactions.aggregate([match, group]):
    print(transaction)


{'_id': 'Debit-Card', 'Total': 410}
{'_id': 'Credit-Card', 'Total': 820}


In [69]:
# Q-4. Trouver l'Id le plus élevé.

tri = {"$sort": {"Id": -1 }}
limit = {"$limit": 1}
for transaction in transactions.aggregate([tri,limit]):
    pprint(transaction)


{'Id': 103,
 'Name': 'Dylan',
 'Note': 'Payment is Null',
 'Panier': [{'ItemId': 'a100', 'price': 200}, {'ItemId': 'a110', 'price': 210}],
 'Payment': None,
 'Subscriber': True,
 'TransactionId': 'tran4',
 '_id': ObjectId('601015ce0c3964a51ae3b528')}


In [73]:
# Q-5. Find the max price (Transaction.price) ?

eclater_liste = {"$unwind" : "$Panier"}
tri = {"$sort": {"Panier.price": -1 }}
limit = {"$limit": 1}
for transaction in transactions.aggregate([tri,eclater_liste,limit]):
    pprint(transaction)



{'Id': 102,
 'Name': 'Margaret',
 'Panier': {'ItemId': 'a100', 'price': 200},
 'Payment': {'Success': True, 'Total': 410, 'Type': 'Credit-Card'},
 'Subscriber': True,
 'TransactionId': 'tran3',
 '_id': ObjectId('601015ce0c3964a51ae3b526')}
